Prepering data:

In [15]:
import pandas as pd
import numpy as np

data1 = pd.read_json("proj3_data1.json")
data2 = pd.read_json("proj3_data2.json")
data3 = pd.read_json("proj3_data3.json")
one_big_df = pd.concat([data1, data2, data3], ignore_index=True)
print(one_big_df.shape)

print(type(one_big_df))

with open("proj3_ex01_all_data.json", "w") as file:
    one_big_df.to_json(file)


(18, 8)
<class 'pandas.core.frame.DataFrame'>


Missing values:

In [16]:
missing = one_big_df.isnull().sum()
missing = missing[missing > 0]

missing.to_csv("proj3_ex02_no_nulls.csv", header=False)

Params dictionary:

In [17]:
import json

with open("proj3_params.json", "r") as file:
    params = json.load(file)

one_big_df["description"] = one_big_df[params["concat_columns"]].astype(str).agg(" ".join, axis=1)

one_big_df.to_json("proj3_ex03_descriptions.json")

More data:

In [18]:
data4 = pd.read_json("proj3_more_data.json")
one_big_df_merged = pd.merge(one_big_df, data4, on="engine", how="left")
one_big_df_merged.to_json("proj3_ex04_joined.json")

Iterating:

In [19]:
for index, row in one_big_df_merged.iterrows():
    file_name = f"proj3_ex05_{row['description'].replace(" ", "_").lower()}.json"
    row = row.drop("description")
    row.to_json(file_name)

for index, row in one_big_df_merged.iterrows():
    file_name = f"proj3_ex05_int_{row['description'].replace(" ", "_").lower()}.json"
    row = row.drop("description")

    for column in params["int_columns"]:
        try:
            if pd.notnull(row[column]):
                row[column] = int(row[column])
            else:
                row[column] = None
        except:
            row[column] = None

    row.to_json(file_name)

Aggregation:

In [20]:
agg_dict = {}

for column, func in params["aggregations"]:
    key = f"{func}_{column}"
    value = getattr(one_big_df_merged[column], func)()
    agg_dict[key] = value

with open("proj3_ex06_aggregations.json", "w") as file:
    json.dump(agg_dict, fp=file)

Grouping:

In [21]:
grouped = one_big_df_merged.groupby(params["grouping_column"])

filtered_groups = grouped.filter(lambda x: len(x) > 1)

mean_df = filtered_groups.groupby(params["grouping_column"]).mean(numeric_only=True)

mean_df.to_csv("proj3_ex07_groups.csv", index=True, header=True)


Reshaping data:

In [22]:
import pickle

pivot_df = one_big_df_merged.pivot_table(
    index=params["pivot_index"],
    columns=params["pivot_columns"],
    values=params["pivot_values"],
    aggfunc="max"
)

with open("proj3_ex08_pivot.pkl", "wb") as file:
    pickle.dump(pivot_df, file)

Widening table:

In [23]:
import pandas as pd

melted_df = pd.melt(
    one_big_df_merged,
    id_vars=params["id_vars"],
    var_name="variable",
    value_name="value"
)

melted_df.to_csv("proj3_ex08_melt.csv", index=False, header=True)


Columns with underscore:

In [ ]:
import pandas as pd
import json

df = pd.read_csv("proj3_statistics.csv")

with open("proj3_params.json") as f:
    params = json.load(f)

pivot_index = params["pivot_index"]
id_col = df.columns[0]

df = df.melt(id_vars=df.columns[0])
df[["brand", "year"]] = df["variable"].str.split("_", expand=True)
df = df.drop(labels="variable", axis=1)
df["index"] = list(zip(df["Country"], df["year"]))
df = df.pivot_table(values="value", index="index", columns="brand")

df.to_pickle("proj3_ex08_stats.pkl")

brand,Audi,BMW,Renault,Volkswagen
index,,,,
"(France, 2019)",20.0,29.0,28.0,36.0
"(France, 2020)",22.0,33.0,26.0,40.0
"(Germany, 2019)",24.0,31.0,17.0,44.0
"(Germany, 2020)",26.0,35.0,15.0,48.0
"(Italy, 2019)",18.0,27.0,31.0,48.0
"(Italy, 2020)",20.0,31.0,29.0,52.0
"(Poland, 2019)",12.0,21.0,22.0,32.0
"(Poland, 2020)",14.0,25.0,20.0,36.0
"(Spain, 2019)",16.0,25.0,19.0,40.0


In [ ]:
import pandas as pd
import json

df = pd.read_csv("proj3_statistics.csv")

with open("proj3_params.json") as f:
    params = json.load(f)

pivot_index = params["pivot_index"]
id_col = df.columns[0]

df_melted = df.melt(id_vars=[id_col], var_name="variable", value_name="value")
df_melted[["brand", "year"]] = df_melted["variable"].str.split("_", expand=True)
df_melted
df_melted["year"] = df_melted["year"].astype(int)
df_melted

desired_order = ["Poland", "Germany", "France", "Spain", "Italy"]

df_pivot = df_melted.pivot_table(index=[id_col, "year"], columns="brand", values="value")

df_pivot = df_pivot.reset_index()

df_pivot[id_col] = pd.Categorical(df_pivot[id_col], categories=desired_order, ordered=True)

df_pivot = df_pivot.sort_values(by=[id_col, "year"])

df_pivot = df_pivot.set_index([id_col, "year"])

df_pivot = df_pivot[["Audi", "BMW", "Volkswagen", "Renault"]]

df_pivot.index.names = [None, None]
df_pivot.columns.name = None


Audi   BMW  Volkswagen  Renault
Poland  2019  12.0  21.0        32.0     22.0
        2020  14.0  25.0        36.0     20.0
Germany 2019  24.0  31.0        44.0     17.0
        2020  26.0  35.0        48.0     15.0
France  2019  20.0  29.0        36.0     28.0
        2020  22.0  33.0        40.0     26.0
Spain   2019  16.0  25.0        40.0     19.0
        2020  18.0  29.0        44.0     17.0
Italy   2019  18.0  27.0        48.0     31.0
        2020  20.0  31.0        52.0     29.0